In [1]:
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, BatchNormalization
import pandas as pd
from keras.models import Sequential
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
import os
from typing import Tuple, List
from tqdm import tqdm

In [2]:
def getImageDirs(root: str = "data"):
    imageDirs = []
    for subDirectory, directory, files in os.walk(root):
        for file in files:
            if file[-4:] == ".jpg":
                path = os.path.join(subDirectory, file)         
                imageDirs.append(path)
    return(imageDirs)

In [3]:
def getDirsAndClasses(root: str, file: str) -> Tuple[List[str], List[int]]:
    imageDirs = []
    classes = []
    line = ""
    with open(root + file, "r") as f:
        for line in tqdm(f):
            imageDir, clazz = line.split()
            imageDirs.append(imageDir)
            classes.append(int(clazz))
    return imageDirs, classes

In [4]:
root = os.getcwd() + "\\Data\\CNR-EXT-150x150"
imageDirs, classes = getDirsAndClasses(root, "\\LABELS\\all.txt")
root = root + "\\PATCHES\\"
data = pd.DataFrame([
            {
                "image": root + filename,
                "class": "free" if clazz == 0 else "busy"
            }
            for filename, clazz in tqdm(zip(imageDirs, classes))
    ])
#Split train/test into seperate dfs
train = data.groupby("class").sample(frac = 0.8)
test = data.drop(train.index).reset_index(drop = True)
train = train.reset_index(drop = True)

144965it [00:00, 842095.23it/s]
144965it [00:00, 1372893.97it/s]


In [5]:
data["class"].value_counts()

busy    79307
free    65658
Name: class, dtype: int64

In [6]:
#Declare data generators and preprocessing
train_datagen = ImageDataGenerator(
    #Augment data with random flips, normalize each sample's input
    vertical_flip = True,
    horizontal_flip = True,
    rescale = 1.0 / 255.0,
    samplewise_std_normalization = True
)
train_generator = train_datagen.flow_from_dataframe(
    directory = None, #none since the df has absolute paths
    dataframe = train,
    x_col = "image",
    y_col = "class",
    validate_filenames = False, #faster for huge datasets
    target_size = (150, 150),
    color_mode = "rgb",
    batch_size = 128,
    class_mode = "binary",
    shuffle = True
)

test_datagen = ImageDataGenerator(
    samplewise_std_normalization = True
)
test_generator = train_datagen.flow_from_dataframe(
    directory = None, #none since the df has absolute paths
    dataframe = test,
    x_col = "image",
    y_col = "class",
    validate_filenames = False,
    target_size = (150, 150),
    color_mode = "rgb",
    batch_size = 128,
    class_mode = "binary",
    shuffle = True
)


C:\Users\music\anaconda3\envs\csc790\lib\site-packages\keras_preprocessing\image\image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 115972 non-validated image filenames belonging to 2 classes.
Found 28993 non-validated image filenames belonging to 2 classes.


C:\Users\music\anaconda3\envs\csc790\lib\site-packages\keras_preprocessing\image\image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [7]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor = "accuracy",
        min_delta = 0.01,
        patience = 3,
        verbose = 1
    )
]

In [8]:
#Build Model
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Convolution2D(filters=96, input_shape=(150,150,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Convolution2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Convolution2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Convolution2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Convolution2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(1))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('sigmoid'))

#Declare Optimizer
opt = keras.optimizers.Adam(learning_rate = 0.001)
AlexNet.compile(loss = "binary_crossentropy", optimizer = opt,  metrics = ["accuracy"])
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 38, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 38, 38, 96)        384       
_________________________________________________________________
activation (Activation)      (None, 38, 38, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 19, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 19, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 19, 256)       1024      
_________________________________________________________________
activation_1 (Activation)    (None, 19, 19, 256)       0

In [9]:
#Fit data
AlexNet.fit(train_generator, callbacks = callbacks, epochs = 100)

Epoch 1/100
907/907 [==============================] - 285s 314ms/step - loss: 0.2645 - accuracy: 0.9586
Epoch 2/100
907/907 [==============================] - 92s 102ms/step - loss: 0.1486 - accuracy: 0.9790
Epoch 3/100
907/907 [==============================] - 94s 104ms/step - loss: 0.1052 - accuracy: 0.9807
Epoch 4/100
907/907 [==============================] - 94s 104ms/step - loss: 0.0744 - accuracy: 0.9853
Epoch 5/100
907/907 [==============================] - 95s 104ms/step - loss: 0.0554 - accuracy: 0.9882
Epoch 00005: early stopping


In [11]:
#Test accuracy
AlexNet.evaluate(test_generator)

227/227 [==============================] - 71s 314ms/step - loss: 0.0560 - accuracy: 0.9859


[0.05600070580840111, 0.9859276413917542]

In [12]:
AlexNet.save("Models/AlexNet")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: Models/AlexNet\assets
